In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sqlite3

# Import TMDB Data

In [248]:
tmdb_movies = pd.read_csv('../data/tmdb.movies.csv.gz')
tmdb_movies.rename(columns={'title': 'primary_title'}, inplace=True)
tmdb_movies.drop(columns=['genre_ids', 'original_language', 'original_title'], inplace=True)
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     26517 non-null  int64  
 1   id             26517 non-null  int64  
 2   popularity     26517 non-null  float64
 3   release_date   26517 non-null  object 
 4   primary_title  26517 non-null  object 
 5   vote_average   26517 non-null  float64
 6   vote_count     26517 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 1.4+ MB


# The Numbers Data

In [249]:
tn_movie_budgets = pd.read_csv('../data/tn.movie_budgets.csv.gz')
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [250]:
tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])
tn_movie_budgets['release_month'] = pd.to_datetime(tn_movie_budgets['release_date']).dt.month_name()
tn_movie_budgets['release_year'] = pd.to_datetime(tn_movie_budgets['release_date']).dt.year

tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str.replace('$', '')
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str.replace(',', '').astype(int) 

tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str.replace('$', '')
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str.replace(',', '').astype(int)

tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str.replace('$', '')
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str.replace(',', '').astype(int)

tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_month      5782 non-null   object        
 7   release_year       5782 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 361.5+ KB


In [251]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
tn_movie_budgets['release_month'] = pd.Categorical(tn_movie_budgets['release_month'], categories=months, ordered=True)
tn_movie_budgets['release_month'].value_counts()

December     745
October      573
August       496
September    493
November     486
June         479
March        470
April        454
July         440
May          407
February     392
January      347
Name: release_month, dtype: int64

In [252]:
tn_movie_budgets.rename(columns={'movie': 'primary_title'}, inplace=True)
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   primary_title      5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_month      5782 non-null   category      
 7   release_year       5782 non-null   int64         
dtypes: category(1), datetime64[ns](1), int64(5), object(1)
memory usage: 322.4+ KB


# Removing zeros


In [253]:
tn_movie_budgets_nonzero = tn_movie_budgets[tn_movie_budgets['domestic_gross'] != 0].copy()

In [254]:
tn_movie_budgets_nonzero.describe()

,id,production_budget,domestic_gross,worldwide_gross,release_year
count,5234.000000,5.234000e+03,5.234000e+03,5.234000e+03,5234.000000
mean,50.346580,3.403348e+07,4.625747e+07,1.007615e+08,2003.308942
std,28.848667,4.296048e+07,7.029651e+07,1.811226e+08,12.619158
min,1.000000,1.100000e+03,3.880000e+02,4.010000e+02,1915.000000
25%,25.000000,6.500000e+06,4.289718e+06,8.142571e+06,1999.000000
50%,50.000000,2.000000e+07,2.198422e+07,3.543844e+07,2006.000000
75%,75.000000,4.500000e+07,5.756598e+07,1.093357e+08,2012.000000
max,100.000000,4.250000e+08,9.366622e+08,2.776345e+09,2019.000000


# Adding Worldwide and Domestic ROI Columns to 'tn_movie_budgets' table

In [255]:
tn_movie_budgets_nonzero['domestic_ROI'] = ((tn_movie_budgets_nonzero['domestic_gross'] - tn_movie_budgets_nonzero['production_budget']) / tn_movie_budgets_nonzero['production_budget']) * 100

tn_movie_budgets_nonzero['worldwide_ROI'] = ((tn_movie_budgets_nonzero['worldwide_gross'] - tn_movie_budgets_nonzero['production_budget']) / tn_movie_budgets_nonzero['production_budget']) * 100


# SQL DB

In [256]:
conn = sqlite3.connect('../data/im.db')
pd.read_sql('SELECT name AS "Table Names" FROM sqlite_master WHERE type="table"', conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [257]:
# Making the IMDB movie_basics table into a pandas df
imdb_movie_basics = pd.read_sql('SELECT * FROM movie_basics', conn)
imdb_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [258]:
# Making the IMDB directors table into a pandas df
imdb_directors = pd.read_sql('SELECT * FROM directors', conn)
imdb_directors.value_counts()

movie_id   person_id
tt2249786  nm4630460    52
           nm1667641    52
           nm3952326    52
           nm3911770    52
           nm3863962    52
                        ..
tt3835080  nm0957772     1
tt3835110  nm7064225     1
tt7083156  nm1962933     1
tt3835134  nm7775933     1
tt4105968  nm1297226     1
Length: 163535, dtype: int64

In [259]:
imdb_movie_akas = pd.read_sql('SELECT * FROM movie_akas', conn)
imdb_movie_akas.drop(columns=['is_original_title'], inplace=True)
imdb_movie_akas.head()

,movie_id,ordering,title,region,language,types,attributes
0,tt0369610,10,Джурасик свят,BG,bg,None,None
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None


In [260]:
imdb_writers = pd.read_sql('SELECT * FROM writers', conn)
imdb_writers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255873 entries, 0 to 255872
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   255873 non-null  object
 1   person_id  255873 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB


In [111]:
imdb_writers_and_directors2 = pd.read_sql("""
            SELECT * 
            FROM movie_ratings
            INNER JOIN directors
                USING(movie_id)
            INNER JOIN writers
                USING(movie_id)

            
            """, conn)

In [112]:
d_e = pd.read_sql("""
SELECT m.averagerating, person_id, b.primary_title
FROM movie_ratings as m
    JOIN (SELECT DISTINCT movie_id, person_id
FROM directors
GROUP BY person_id
HAVING COUNT() >5) as d_n_e
    ON d_n_e.movie_id = m.movie_id
    JOIN movie_basics AS b
        USING(movie_id)
""", conn)
d_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7081 entries, 0 to 7080
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   averagerating  7081 non-null   float64
 1   person_id      7081 non-null   object 
 2   primary_title  7081 non-null   object 
dtypes: float64(1), object(2)
memory usage: 166.1+ KB


In [17]:
# using left join
imdb_writers_and_directors.info()

NameError: name 'imdb_writers_and_directors' is not defined

In [261]:
# using inner join
imdb_writers_and_directors2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46166010 entries, 0 to 46166009
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   movie_id       object 
 1   averagerating  float64
 2   numvotes       int64  
 3   person_id      object 
 4   person_id      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.7+ GB


In [262]:
pd.read_sql("""
            SELECT 
            FROM movie_basics AS m
            JOIN directors AS d
                USING(movie_id)
            JOIN writers AS w
                USING(movie_id)

            
            """, conn)

DatabaseError: Execution failed on sql '
            SELECT 
            FROM movie_basics AS m
            JOIN directors AS d
                USING(movie_id)
            JOIN writers AS w
                USING(movie_id)

            
            ': near "FROM": syntax error

In [263]:
movie_akas = pd.read_sql("""

SELECT *
       
FROM movie_akas





""", conn)

In [264]:
#Renaming 'movie' column to 'primary_title' so I can join table with 'tn_movie_budgets' table
movie_akas = movieakas.rename(columns={"movie": "primary_title"})

In [265]:
movie_akas = movie_akas.dropna(subset=['attributes'])


# Combining Dataframes


In [266]:
# Combing the IMDB 'movie_basics' table with the TMDB 'tmdb_movies' table

merged_movie_basics = pd.merge(imdb_movie_basics, tmdb_movies, on='primary_title')
merged_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23612 entries, 0 to 23611
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         23612 non-null  object 
 1   primary_title    23612 non-null  object 
 2   original_title   23607 non-null  object 
 3   start_year       23612 non-null  int64  
 4   runtime_minutes  21750 non-null  float64
 5   genres           23263 non-null  object 
 6   Unnamed: 0       23612 non-null  int64  
 7   id               23612 non-null  int64  
 8   popularity       23612 non-null  float64
 9   release_date     23612 non-null  object 
 10  vote_average     23612 non-null  float64
 11  vote_count       23612 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 2.3+ MB


In [267]:
merged_exp_directors = pd.merge(d_e, tmdb_movies, on='primary_title')
merged_exp_directors = pd.merge(merged_exp_directors, tn_movie_budgets, on='primary_title')
merged_exp_directors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475 entries, 0 to 474
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   averagerating      475 non-null    float64       
 1   person_id          475 non-null    object        
 2   primary_title      475 non-null    object        
 3   Unnamed: 0         475 non-null    int64         
 4   id_x               475 non-null    int64         
 5   popularity         475 non-null    float64       
 6   release_date_x     475 non-null    object        
 7   vote_average       475 non-null    float64       
 8   vote_count         475 non-null    int64         
 9   id_y               475 non-null    int64         
 10  release_date_y     475 non-null    datetime64[ns]
 11  production_budget  475 non-null    int64         
 12  domestic_gross     475 non-null    int64         
 13  worldwide_gross    475 non-null    int64         
 14  release_mo

In [268]:
#Combining SQL table 'movie_akas' with 'tn_movie_budgets'
movie_akas_budgets = pd.merge(movie_akas, tn_movie_budgets_nonzero, on = 'primary_title')
#Combining the combined dataframe 'movie_akas_budgets' with 'tmbd_movies' to get the popualarity and vote ratings
movieakas_budgets_popularity = pd.merge(movie_akas_budgets, tmdb_movies, on = 'primary_title')

In [269]:
movieakas_budgets_popularity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 317
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie_id           318 non-null    object        
 1   ordering           318 non-null    int64         
 2   primary_title      318 non-null    object        
 3   region             317 non-null    object        
 4   language           100 non-null    object        
 5   types              0 non-null      object        
 6   attributes         318 non-null    object        
 7   is_original_title  318 non-null    float64       
 8   id_x               318 non-null    int64         
 9   release_date_x     318 non-null    datetime64[ns]
 10  production_budget  318 non-null    int64         
 11  domestic_gross     318 non-null    int64         
 12  worldwide_gross    318 non-null    int64         
 13  release_month      318 non-null    category      
 14  release_ye

In [289]:
#Dropping columns from the merged "movie_akas_budgets" data frame that we will not need.
movieakas_budgets_popularity.drop(columns=['movie_id', 'ordering', 'region', 'language', 'types', 'Unnamed: 0', 'is_original_title', 'id_x', 'release_month', 'release_year', 'id_y', 'release_date_y'], inplace=True)

KeyError: "['movie_id' 'ordering' 'region' 'language' 'types' 'Unnamed: 0'\n 'is_original_title' 'id_x' 'release_month' 'release_year' 'id_y'\n 'release_date_y'] not found in axis"

# Creating Tables With and Without 3-D Movies

In [272]:
threed_budgets_df= movieakas_budgets_popularity[movieakas_budgets_popularity['attributes'].str.contains('3-D')]
no_threed_budgets_df= movieakas_budgets_popularity[movieakas_budgets_popularity['attributes'].str.contains('3-D')==False]

In [286]:
#Dropping duplicate rows if release date and title match. Also including worldwide_gross as a criteria to make sure we are not deleting different releases
threed_budgets_df = threed_budgets_df.drop_duplicates(subset=['release_date_x', 'primary_title', 'worldwide_gross'])

no_threed_budgets_df = no_threed_budgets_df.drop_duplicates(subset=['release_date_x', 'primary_title', 'worldwide_gross'])